# Klasifikácia pomocou rozhodovacích stromov

Rozhodovacie stromy patria medzi populárne metódy klasifikácie. Medzi podstatné vlastnosti, kvôli ktorým sú klasifikátory na báze rozhodovacích stromov obľúbené, je ich interpretovateľnosť. Stromy predstavujú dobre pochopiteľnú a zároveň prezentovateľnú štruktúru, ktorá môže byť vhodná, ak je potrebné vnútornú štruktúru modelu (a teda aj spôsob, ako model "dospel" k danému výsledku) vysvetliť alebo prezentovať. 

Klasifikátor rozhodovacích stromov je v knižnici Scikit-learn implementovaný triedou `DecisionTreeClassifier`. 

Aj napriek faktu, že mnoho algoritmov pre indukciu rozhodovacích stromov je schopných pracovať s kategorickými atribútmi, implementácia tohoto klasifikátora v Scikit-learn bohužiaľ s kategorickými premennými pracovať nevie. Dáta je preto nutné predspracovať, transformovaním kategorických atribútov na numerické. Odporúča sa na všetky atribúty bez usporiadania transformovať pomocou One Hot Encoderu. 

Okrem toho, z datasetu v prípade rozhodovacích stromov môžeme odstrániť irelevantné a redundantné atribúty. V prípade datasetu Titanic teda môžeme odstrániť atribúty `age_ordinal` a `fare_ordinal`, keďže vznikli odvodením z existujúcich atribútov `age` a `fare`. 

Rozhodovacie stromy ale nevyžadujú normalizáciu atribútov.

In [ ]:
# Titanic import a preprocessing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


titanic = pd.read_csv("../data/titanic-processed.csv")

titanic = titanic.drop(columns=['cabin','ticket','title','deck','fare_ordinal','age_ordinal'])

titanic['sex'] = titanic['sex'].map({"male": 0, "female": 1})
titanic['has_family'] = titanic['has_family'].map({False: 0, True: 1})

titanic = pd.get_dummies(titanic, columns=['embarked', 'title_short'])

titanic.head()

In [ ]:
X_titanic = titanic.drop('survived', axis=1) # vytvoríme maticu príznakov - použijeme všetky stĺpce okrem cieľového atribútu a uložíme do X_titanic
y_titanic = titanic['survived'] # vytvoríme vektor hodnôt cieľového atribútu ako stĺpec 'survived'

print(X_titanic.shape) # pre kontrolu môžeme vypísať rozmery matice hodnôt a vektora cieľového atribútu
print(y_titanic.shape)

from sklearn.model_selection import train_test_split # importujeme funkciu train_test_split()
X_train, X_test, y_train, y_test = train_test_split(X_titanic, y_titanic, test_size=0.3, random_state=1) # rozdelíme dataset do trénovacej a testovacej časti, tak že testovacia bude 30% z celkového datasetu

Model klasifikátora na báze rozhodovacích stromov potom trénujeme podobne, ako v prípade modelu k-NN. Použijeme triedu `DecisionTreeClassifier`, inicializujeme model (prípadne nastavíme parametre modelu) a model natrénujeme na trénovacích dátach. 

Pri učení stromového modelu môžeme model nastavovať nasledovými parmetrami:
* criterion - kritérium pre výber atribútu: "gini" alebo "entropy"
* max_depth - maximálna hĺbka stromu (ak je nastavená na None, expanduje sa kompletný strom)
* min_samples_split - najmenší počet príkladov potrebných pre vetvenie uzlu
* min_samples_leaf - najmenší možný počet príkladov v listovom uzle
* presort - True/False - utriedenie atribútov pre urýchlenie trénovania

In [ ]:
from sklearn.tree import DecisionTreeClassifier # Importovanie triedy zodpovedajúcej modelu, ktorý budeme trénovať
from sklearn.metrics import confusion_matrix

dt = DecisionTreeClassifier()   # Inicializácia stromového modelu   
dt.fit(X_train, y_train)        # Trénovanie modelu na trénovacej množine 
y_dt = dt.predict(X_test)       # Otestovanie modelu na testovacej množine

from sklearn.metrics import accuracy_score,precision_score, recall_score # vypočítanie metrík kvality modelu

print(f"Presnosť (accuracy) modelu: {accuracy_score(y_test, y_dt)}")
print(f"Presnosť (precision) modelu: {precision_score(y_test, y_dt)}")
print(f"Návratnosť (recall) modelu: {recall_score(y_test, y_dt)}")

cm = confusion_matrix(y_test, y_dt)  # vypísanie kontigenčnej tabuľky
print(cm)

#### Zobrazenie stromového modelu

Ako sme na úvod spomínali, stromový model je možné vizualizovať, čo je podstatné pre pochopenie modelu a spôsobu jeho fungovania. Na druhej strane - podstatný faktor je ale zložitosť samotného modelu. Priveľmi rozvetvené stromy, s veľmi bohatou štruktúrou sú veľmi ťažko čitateľné a neprehľadné, čím strácajú tento benefit. 

Skúsme sa pozrieť, asi ako vyzerá rozhodovací strom pre implicitne nastavený stromový klasifikátor natrénovaný na trénovacích dátach datasetu Titanic.

V jazyku Python a knižnici Scikit-learn existuje niekoľko spôsobov, ako stromové modely vizualizovať. Väčšina zahŕňa inštaláciu externých progamov ako GraphViz, resp. rôznych ďalších modulov, preto pre účely cvičenia použijeme iba export stromu do súboru v GraphViz formáte. Vytvorený súbor (súbor s príponou `.dot`) môžeme potom otvoriť a strom zobraziť vo webovej verzii aplikácie GraphViz. Tá je dostupná na www.webgraphivz.com.

Na export natrénovaného rozhodovacieho stromu použijeme funkciu Scikit-learn `export_graphviz()`. Tej špecifikujeme ako parameter stromový model, ktorý chceme vykresliť, `feature_names` obsahujúci zoznam hlavičiek atribútov (kvôli vykresľovaniu uzlov), `class_names` obsahujúci zoznam hodnôt cieľového atribútu a výstupný súbor, do ktorého vizualizáciu uložíme.

In [ ]:
from sklearn import tree
from sklearn.tree import export_graphviz

with open("decision_tree.txt", "w") as f:
    f = tree.export_graphviz(dt, feature_names=X_titanic.columns.values, class_names=['0','1'], out_file=f)

Po spustení tohoto odstavca uvidíte v prehliadači súborov Jupyter Labu vľavo súbor `decision_tree.txt`. Otvorte ho (dá sa otvoriť aj priamo v Jupyter prostredí) a skopírujte jeho kompletný obsah do okna webovej aplikácie www.webgraphviz.com. V nej po stlačení tlačidla Generate Graph vygenerujete vizualizáciu rozhodovacieho stromu. Preskúmajte ju. 

#### Úloha 12.2.

Ak je výsledný strom príliš zložitý a nečitateľný - ktorý z parametrov by ste použili a s akou hodnotou, aby ste vygenerovali všeobecnejší a prehľadnejší strom? Upravte kód trénovania modelu a natrénujte model s identifikovaným a nastaveným parametrom a natrénujte takýto model. Potom ho vizualizujte a porovnajte s modelom natrénovaným bez nastavení parametrov.

### Reprezentácia modelu pomocou pravidiel

Rozhodovacie stromy okrem vizualizácie umožňujú odlišný spôsob prezentácie štruktúry modelu - a to tak, že zo štruktúry stromu vygenerujeme pravidlá. Takéto pravidlá môžu byť v tvare `if` podmienka `then` záver. Môžu byť extrahované priamo zo štruktúry stromu a zodpovedajú jednotlivým testom (podmienka) a vetveniam (závery). 

Nižšie je kód funkcie `tree_to_code`, ktorú môžeme použiť na pretransformovanie stromovej štruktúry do pravidiel. Funkcia obsahuje implementované odsadenie vypisovania textu časti pravidiel pre zlepšenie citateľnosti výstupu.

Spustite funkciu na vytvorenom stromovom modeli a porovnajte extrahované pravidlá so stromovou štruktúrou:

In [ ]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names): 

	tree_ = tree.tree_
	feature_name = [
		feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
		for i in tree_.feature
	]
	print("def tree({}):".format(", ".join(feature_names)))

	def recurse(node, depth):
		indent = "  " * depth
		if tree_.feature[node] != _tree.TREE_UNDEFINED:
			name = feature_name[node]
			threshold = tree_.threshold[node]
			print("{}if {} <= {}:".format(indent, name, threshold))
			recurse(tree_.children_left[node], depth + 1)
			print("{}else:  # if {} > {}".format(indent, name, threshold))
			recurse(tree_.children_right[node], depth + 1)
		else:
			print("{}return {}".format(indent, tree_.value[node]))

	recurse(0, 1)

In [ ]:
# zavoláme funkciu tree_to_code na vytvorenom modeli dt, názvy atribútov sú X_titanic.column.values

tree_to_code(dt, X_titanic.columns.values)

Dôležitosť atribútov pre klasifikáciu vieme spočítať aj separátne. Môžeme na to použiť napr. funkciu SelectKBest, pomocou ktorej vieme odfiltrovať k najpodstatnejších atribútov pre tvorbu modelu. Vieme ju takisto použiť pre spočítanie dôležitosti atribútov a selekciu spraviť manuálne. 

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

fs = SelectKBest(score_func=mutual_info_classif, k='all')
fs.fit(X_train, y_train)

Dôležitosť atribútov potom vieme vypísať na obrazovku alebo vykresliť do grafu. 

In [ ]:
for i in range(len(fs.scores_)):
	print('Atribut %d: %f' % (i, fs.scores_[i]))
    
    
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()

#### Úloha 12.3.

Použite Grid Search pre nájdenie optimálnej kombinácie parametrov modelu rozhodovacieho stromu na dátach Titanic. V Grid Search použite 5-násobnú krížovú validáciu a ako metriku vyhodnotenia použite `accuracy`. Identifikujte najlepší model vypíšte preň metriky presnosti a návratnosti (vypíšte aj kontigenčnú tabuľku - confusion matrix).

In [ ]:
# YOUR CODE HERE

#### Úloha 12.4.

Z výsledkov Grid Search zistite a vykreslite vzájomnú závislosť presnosti modelu od parametra udávajúceho maximálnu hĺbku stromu. Na vizualizáciu použite matplotlib. 

In [ ]:
# YOUR CODE HERE